In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_excel("demo_data.xlsx",sheet_name="demo_claims")
df.count()
df.columns=df.columns.str.lower()

In [3]:
count_unique= df.nunique().to_list()
card_perc= [i[0]/i[1] for i in zip(count_unique, df.count())]
cardinality=list(zip(df.columns,count_unique, df.count(),card_perc))
print(cardinality)

[('memberid', 5, 17, 0.29411764705882354), ('claimid', 8, 17, 0.47058823529411764), ('providerid', 5, 17, 0.29411764705882354), ('providername', 5, 17, 0.29411764705882354), ('benefithead', 8, 17, 0.47058823529411764), ('benefitdescr', 8, 17, 0.47058823529411764), ('procedurecode', 8, 13, 0.6153846153846154), ('treatmentcountry', 2, 17, 0.11764705882352941), ('amount', 13, 17, 0.7647058823529411), ('currencycode', 2, 17, 0.11764705882352941), ('conversionrate', 2, 17, 0.11764705882352941), ('amountgbp', 15, 17, 0.8823529411764706)]


In [5]:
dims=[]
def dim_from_flat(df, key_name, attributes):
    dim_fields=[key_name]
    for i in attributes:
        df_unique=df[[key_name,i]].drop_duplicates(ignore_index=True)
        df_check_key_unique=df_unique[df_unique[key_name].duplicated(keep=False)]
        if len(df_check_key_unique)==0:
            print(i," can be included")
            dim_fields.append(i)
        else:
            print(i," cannot be included as it has instances of multiple values per key ", key_name)
            print(df_check_key_unique.head())
            
    df_dim=df[dim_fields].drop_duplicates(ignore_index=True)
    df_dim.insert(0, "dimkey_"+key_name, range(1, 1 + len(df_dim)))
    df_dim.reset_index(drop=True,inplace=True)
    df_fact=df.merge(df_dim[[key_name,"dimkey_"+key_name]],how='left',left_on=key_name, right_on=key_name)
    df_fact.drop(dim_fields, axis=1, inplace=True)
    return df_dim, df_fact


df_dim,df_fact= dim_from_flat(df_new,"currencycode",["conversionrate",'claimid'])
df_fact.head()

conversionrate  can be included
claimid  cannot be included as it has instances of multiple values per key  currencycode
  currencycode claimid
0          USD     C01
1          USD     C02
2          GBP     C02
3          GBP     C03
4          USD     C03


,memberid,claimid,providerid,providername,benefithead,benefitdescr,procedurecode,treatmentcountry,amount,amountgbp,dimkey_currencycode
0,M01,C01,P01,Clinic Mayo,10,Pregnancy Checks,901.0,US,1000,783.249121,1
1,M01,C02,P01,Clinic Mayo,10,Pregnancy Checks,902.0,US,800,626.599297,1
2,M01,C02,P02,Pharmacy Excel,20,Pain relief/anti inflammatory,NaN,UK,400,400.000000,2
3,M02,C03,P03,Dr Grim,30,GP Diagnosis,501.0,UK,200,200.000000,2
4,M02,C03,P01,Clinic Mayo,10,Pregnancy Checks,901.0,US,1300,1018.223857,1


In [6]:
df_dim

,dimkey_currencycode,currencycode,conversionrate
0,1,USD,1.276733
1,2,GBP,1.000000


In [7]:
df_fact.shape

(17, 11)